In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12
import numpy as np
import galois 
GF = galois.GF(curve_order)

In [4]:
x = GF(5)
y = GF(10)

def mod_array(a):
    return np.array([mod_scalar(u) for u in a.flatten()]).reshape(a.shape)

def mod_scalar(a):
    return int(a % curve_order)

def ecmul(point, scalar):
    return multiply(point, mod_scalar(scalar))

def ecadd(s, *args):
    for x in args:
        if isinstance(x, tuple) and len(x) == 2:
            # Directly add if x is a point on the curve
            s = add(s, x)
        else:
            # Assume x is a scalar and try multiplying with G1
            try:
                s = add(s, ecmul(G1, x))
            except Exception:
                # If it fails, try with G2
                s = add(s, ecmul(G2, x))
    return s

L = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,-5,0,0,0,0,0]
    ])

R = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ])

O = np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [-2,1,0,3,0,-1,0,-2]
    ])

r1cs_m, r1cs_n  = L.shape[1], L.shape[0] #m columns, n rows

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"

L_galois = GF(mod_array(L))
R_galois = GF(mod_array(R))
O_galois = GF(mod_array(O))

# QAP
def interpolate_columns(col):
    xs = GF(np.array([1, 2, 3, 4]))
    return galois.lagrange_poly(xs, col)

def matrix_to_polys(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def print_polys(name, polys):
    print(name, "polys:")
    for poly in polys:
        print("  ", poly)

def mul_polys_by_witness(polys, witness):
    return [poly * int(witness[i]) for i, poly in enumerate(polys)]


U_polys, V_polys, W_polys = matrix_to_polys(L_galois), matrix_to_polys(R_galois), matrix_to_polys(O_galois)



# Trusted Setup
tau = GF(9)
alpha = GF(10)
beta = GF(15)
gamma = GF(21)
delta = GF(5)
gamma_inv = np.reciprocal(gamma)
delta_inv = np.reciprocal(delta)

alphag1 = ecmul(G1, alpha)
betag1 = ecmul(G1, beta)
betag2 = ecmul(G2, beta)
deltag1 = ecmul(G1, delta)
deltag2 = ecmul(G2, delta)




r = GF(3)
s = GF(4)




m=8, n=5
